In [1]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import sqlite3

In [2]:
def calc_cosine_sim(model_name, sentence_src, sentence_target):
    model = SentenceTransformer(model_name)
    embedding_src = model.encode(sentence_src, convert_to_tensor=True)
    embedding_target = model.encode(sentence_target, convert_to_tensor=True)
    cosine_score = util.pytorch_cos_sim(embedding_src, embedding_target)
    return cosine_score

In [55]:
conn = sqlite3.connect("../data/clean/correspondence.db")
sql_query = "SELECT * FROM correspondence"
df_corr = pd.read_sql_query(sql_query, conn)
df_corr["name_external"] = df_corr["name_external"] + ", " + df_corr["source_external"]
df_corr.head()
# df_exio2fao = df_exio2fao.drop_duplicates(subset=["Item"], keep="first").reset_index(drop=True)

,name_bonsai,name_external,source_external
0,agave fibres nes,"agave fibres nes, food and agriculture organiz...",food and agriculture organization corporate st...
1,"alcohol, non-food purposes","alcohol, non-food purposes, food and agricultu...",food and agriculture organization corporate st...
2,"almonds, shelled","almonds, shelled, food and agriculture organiz...",food and agriculture organization corporate st...
3,almonds,"almonds, food and agriculture organization cor...",food and agriculture organization corporate st...
4,"anise, badian, fennel","anise, badian, fennel, food and agriculture or...",food and agriculture organization corporate st...


In [72]:
arr_bonsai= df_corr["name_bonsai"].values
arr_external= df_corr["name_external"].values

# cosine_score = calc_cosine_sim("all-mpnet-base-v2", arr_external, arr_bonsai)
# cosine_score = calc_cosine_sim("all-MiniLM-L6-v2", arr_external, arr_bonsai)

Encoder Models:
- multi-qa-mpnet-base-dot-v1
- all-MiniLM-L6-v2
- all-mpnet-base-v2

In [86]:
def match_classification(arr_external, arr_bonsai, model_name):
    cosine_score = calc_cosine_sim(model_name, arr_external, arr_bonsai)
    sorted_cs, indices = cosine_score.sort(dim=1, descending=True)
    result_df = pd.DataFrame()
    
    for ix, product in enumerate(arr_external):
        sorted_product_cs = sorted_cs[ix].cpu().numpy()
        exio_ix = indices[ix].cpu().numpy()
        result_df.loc[ix, "name_external"] = product
        result_df.loc[ix, "name_bonsai_"+model_name] = df_corr.loc[exio_ix[0], "name_bonsai"]
        result_df.loc[ix, 'cosine_score'] = float("{:.3f}".format(sorted_product_cs[0]))
    
    df_merge = result_df.merge(
        df_corr, 
        on=["name_external"], 
        how="right",
        ).drop_duplicates()
    df_merge["compare"] = df_merge.apply(lambda row: row["name_bonsai_"+ model_name] == row["name_bonsai"], axis=1)
    print(len(df_merge[df_merge["compare"]==True])/len(df_merge))
    return df_merge

In [87]:
df_merge = match_classification(arr_external, arr_bonsai, "multi-qa-mpnet-base-dot-v1")

0.5581818181818182


In [77]:
len(df_merge), len(df_corr), len(result_df)
df_merge.head()

,name_external,name_bonsai_nlp,cosine_score,name_bonsai,source_external
0,"agave fibres nes, food and agriculture organiz...",agave fibres nes,0.765,agave fibres nes,food and agriculture organization corporate st...
1,"alcohol, non-food purposes, food and agricultu...","alcohol, non-food purposes",0.693,"alcohol, non-food purposes",food and agriculture organization corporate st...
2,"almonds, shelled, food and agriculture organiz...","almonds, shelled",0.717,"almonds, shelled",food and agriculture organization corporate st...
3,"almonds, food and agriculture organization cor...",almonds,0.697,almonds,food and agriculture organization corporate st...
4,"anise, badian, fennel, food and agriculture or...","anise, badian, fennel",0.703,"anise, badian, fennel",food and agriculture organization corporate st...


In [35]:
df_merge.sort_values(by="cosine_score", ascending=True).head(10)

,name_external,name_bonsai_nlp,cosine_score,name_bonsai,source_external,compare
473,"aquavit, 40 % vol., average values","juice, orange, single strength",0.407,"beverages, distilled alcoholic",concito,False
1093,"remoulade, average values","peppermint, spearmint",0.445,"eggs, hen, in shell",concito,False
1095,"remoulade, average values","peppermint, spearmint",0.445,"oil, sunflower",concito,False
1359,"water, tap, drinking, average values",steam and hot water supply services,0.448,food prep nes,concito,False
731,"dumplings, average values",cake of kapok,0.459,pastry,concito,False
...,...,...,...,...,...,...
737,energy drink,"beverages, distilled alcoholic",0.584,food prep nes,concito,False
1346,"vegan nuggets, soy based",soybeans,0.584,soya paste,concito,False
617,"celeriac, celery root, raw",lettuce and chicory,0.584,roots and tubers nes,concito,False
1272,"spring roll, frozen, ready meals","oats, rolled",0.585,fonio,concito,False


In [36]:
df_merge[df_merge["compare"]==False]

,name_external,name_bonsai_nlp,cosine_score,name_bonsai,source_external,compare
52,buffaloes,buffalo meat,0.756,buffalos - meat (live),food and agriculture organization corporate st...,False
73,cattle,"meat, cattle",0.834,cattle - meat (live),food and agriculture organization corporate st...,False
74,"cow milk, whole (fresh)","milk, whole fresh cow",0.950,cattle - milk,food and agriculture organization corporate st...,False
187,goats,goat meat,0.768,goats - meat (live),food and agriculture organization corporate st...,False
339,camels,fat of camels,0.755,other animal products,food and agriculture organization corporate st...,False
...,...,...,...,...,...,...
1420,geothermal,natural gas liquids,0.495,electricity,international energy agency,False
1422,solar thermal,sunflower seed,0.489,electricity,international energy agency,False
1423,"""tide, wave and ocean""",snails o/t sea snails,0.468,electricity,international energy agency,False
1425,other sources,figs,0.459,electricity,international energy agency,False
